## Quantifying localization-based clustering performance
__Keith Cheveralls__<br>
__October 2021__

This notebook documents how the 'performance' of the localization-based Leiden clustering of OpenCell targets was quantified, particularly the calculation of the Adjusted Rand Index (ARI) as a function of the Leiden clustering resolution. 

In [ ]:
import anndata as ad
import datetime
import numpy as np
import pandas as pd
import pathlib
import scanpy as sc
import seaborn as sns
import sys

import sklearn.manifold
import sklearn.metrics
import sklearn.neighbors

from matplotlib import pyplot as plt
from matplotlib import rcParams

In [ ]:
%load_ext autoreload
%autoreload 1

sys.path.append('../../')
%aimport scripts.cytoself_analysis.clustering_workflows
%aimport scripts.cytoself_analysis.ground_truth_labels
%aimport scripts.cytoself_analysis.go_utils
from scripts.cytoself_analysis import (
    clustering_workflows, ground_truth_labels, go_utils
)

sc.settings.set_figure_params(dpi=80, facecolor='white', frameon=False)
rcParams['font.family'] = 'sans-serif'
rcParams['axes.grid'] = False

data_dir = pathlib.Path('../../data')
output_dir = pathlib.Path(
    '/Users/keith.cheveralls/Box/KC-opencell-paper/image-based-clustering/'
)

def timestamp():
    return datetime.datetime.now().strftime('%Y-%m-%d')

### Load the adata object of target localization encodings
This anndata object includes results of preprocessing: the 200 PCs, kNN matrix, and UMAP coords. The generation of this object is documented in the notebook `generate-localization-encoding.ipynb` and it can be downloaded from figshare [here](https://figshare.com/articles/dataset/Consensus_protein_localization_encodings_for_all_OpenCell_targets/16754965). 

In [ ]:
adata = ad.read_h5ad(data_dir / 'figshare' / 'final-opencell-target-localization-encodings.h5ad')
cwv = clustering_workflows.ClusteringWorkflow(adata=adata)

In [ ]:
sc.pl.umap(cwv.adata, color='grade_3_annotation', palette='tab10', alpha=0.5)

## Plots of ARI vs Leiden resolution

Plot the ARI of the leiden clustering as a function of the resolution parameter, using a variety of ground-truth clustering datasets (particularly all of the grade-3 OC annotations, Kegg pathways, and CORUM complexes).

In [ ]:
# number of random seeds for the Leiden clustering over which to average the ARI
n_random_states = 20

#### Using CORUM

In [ ]:
# largest 5 clusters are those larger than 50 targets; largest 15 are those larger than 30
ground_truth_labels.merge_corum(cwv.adata.obs, drop_largest=15)

In [ ]:
ari_corum_all = cwv.calculate_ari(
    ground_truth_label='corum_labels', n_random_states=n_random_states
)

In [ ]:
ari_corum_wo_largest = cwv.calculate_ari(
    ground_truth_label='corum_labels_wo_largest', n_random_states=n_random_states
)

In [ ]:
# the number of targets in any CORUM cluster and in only one cluster
(
    (cwv.adata.obs.corum_labels != 'none').sum(),
    (
        (cwv.adata.obs.corum_labels != 'none') & 
        (cwv.adata.obs.corum_labels.str.split(';').apply(len) == 1)
    ).sum()
)

#### Kegg pathways

In [ ]:
cwv.adata.obs = ground_truth_labels.merge_kegg_pathways(cwv.adata.obs)
ari_kegg_pathway = cwv.calculate_ari(
    ground_truth_label='pathway_id', n_random_states=n_random_states
)

#### OpenCell annotations labels

In [ ]:
# merge opencell annotations (the ocgt_label column)
cwv.adata.obs = ground_truth_labels.merge_opencell_annotations(
    cwv.adata.obs, data_dir / '2021-06-09-all-opencell-lines.json', only_count_grade_3=True
)

In [ ]:
# using our opencell ground-truth (single grade-3 annotations)
ari_ocgt = cwv.calculate_ari(ground_truth_label='ocgt_label', n_random_states=n_random_states)

In [ ]:
# count OC labels
pd.concat(
    (cwv.adata.obs.ocgt_label.value_counts(), cwv.adata.obs.grade_3_annotation.value_counts()),
    axis=1
)

#### Plot OC, Kegg, CORUM ARIs

In [ ]:
blue, orange, green, red, *_ = sns.color_palette('tab10')

plt.figure(figsize=(8, 6))
plt.gca().set_xlabel('Leiden resolution')
plt.gca().set_ylabel('Adjusted rand index')

x, y = 'resolution', 'ari'

sns.lineplot(data=ari_ocgt, x=x, y=y, label='OpenCell annotations', color=blue)
sns.lineplot(data=ari_kegg_pathway, x=x, y=y, label='Kegg pathways', color=green)
sns.lineplot(data=ari_corum_all, x=x, y=y, label='CORUM clusters', color=red)
sns.lineplot(data=ari_corum_wo_largest, x=x, y=y, label='CORUM clusters (w/o largest)', color=orange)

# plot the median cluster size on the right-hand y-axis
if True:
    ax2 = plt.gca().twinx()
    sns.lineplot(data=ari_ocgt, x=x, y='median_cluster_size', ax=ax2, color='gray')
    ax2.set(xscale='log')
    ax2.set(yscale='log')
    ax2.set_ylabel('Number of clusters')

plt.gca().set(xscale='log')
plt.title('ARI for targets in only one ground-truth cluster')

plt.savefig(
    output_dir / ('%s-clustering-performance--ari-vs-leiden.pdf' % (timestamp(),)),
    bbox_inches='tight'
)

#### Calculate the optimal resolutions for each ground-truth dataset

In [ ]:
sc.tl.umap(cwv.adata, init_pos='spectral', min_dist=0.0, random_state=51)

In [ ]:
def argmax(bm):
    ari_mean = bm.groupby('resolution').mean()
    return ari_mean.iloc[ari_mean.ari.argmax()].name

In [ ]:
argmax(ari_ocgt), argmax(ari_kegg_pathway), argmax(ari_corum_all), argmax(ari_corum_wo_largest)

### Aside: ARI curves using GO-slim and HPA labels
This is unused.

In [ ]:
ground_truth_labels.merge_go_slim(cwv.adata.obs)

In [ ]:
cwv.adata.obs['go_molecular_function'].str.split(';').explode().value_counts()

In [ ]:
ari_go_cc = cwv.calculate_ari(ground_truth_label='go_cellular_component', n_random_states=3)
ari_go_bp = cwv.calculate_ari(ground_truth_label='go_biological_process', n_random_states=3)
ari_go_mf = cwv.calculate_ari(ground_truth_label='go_molecular_function', n_random_states=3)

In [ ]:
# using HPA labels
cwv.adata.obs = ground_truth_labels.merge_hpa_labels(cwv.adata.obs)
ari_hpa = cwv.calculate_ari(ground_truth_label='hpa_main_location', n_random_states=9)

In [ ]:
plt.figure(figsize=(8, 6))
plt.gca().set_xlabel('Leiden resolution')
plt.gca().set_ylabel('Adjusted rand index')

x, y = 'resolution', 'ari'

sns.lineplot(data=ari_ocgt, x=x, y=y, label='OC grade-3 labels')
sns.lineplot(data=ari_go_cc, x=x, y=y, label='GO-Slim CC')
sns.lineplot(data=ari_go_bp, x=x, y=y, label='GO-Slim BP')
sns.lineplot(data=ari_go_mf, x=x, y=y, label='GO-Slim MF')

ax2 = plt.gca().twinx()
sns.lineplot(data=ari_ocgt, x=x, y='median_cluster_size', ax=ax2, color='gray')

# this also sets the x-axis of left axis to log-scale
ax2.set(xscale='log')

ax2.set(yscale='log')
ax2.set_ylabel('Median cluster size')

plt.title('ARI for targets in only one ground-truth cluster')

In [ ]:
ari_corum_all.groupby('resolution').mean()

### Aside: simulate an ARI curve by partially shuffling ground-truth labels

In [ ]:
def ari_for_shuffled_labels(labels, n):
    labels_unshuffled = labels.copy()
    np.random.shuffle(labels_unshuffled)
    
    labels_shuffled = labels_unshuffled.copy()
    np.random.shuffle(labels_shuffled)

    labels_mixed = np.concatenate((labels_unshuffled[:n], labels_shuffled[n:]), axis=0)
    return sklearn.metrics.adjusted_rand_score(labels_unshuffled, labels_mixed),

In [ ]:
d = cwv.adata.obs.copy()
labels = d['cluster_id_leiden_res3_seed18'].values

labels = d.ocgt_label.values
labels = labels[labels != 'none'].copy()

sim_aris = []
for n in np.arange(0, len(labels), 100):
    ari = np.mean([ari_for_shuffled_labels(labels, n) for _ in range(2)])
    sim_aris.append((n, ari))

In [ ]:
sim_aris = np.array(sim_aris)
plt.plot(sim_aris[:, 0]/len(labels), sim_aris[:, 1])

In [ ]:
sim_aris = np.array(sim_aris)
plt.plot(sim_aris[:, 0]/len(labels), sim_aris[:, 1])

### Enrichment of Kegg pathways in each cluster (at the kegg-optimized clustering resolution)

In [ ]:
# kegg pathway_ids and cluster_ids at the optimal resolution for kegg
df = cwv.adata.obs[['ensg_id', 'pathway_id', 'cluster_id_leiden_res6.31_seed44']].copy()
df.rename(
    columns={
        'pathway_id': 'label_id'
        'cluster_id_leiden_res6.31_seed44': 'cluster_id', 
    }, 
    inplace=True
)

In [ ]:
df['label_id'] = df.label_id.str.split(';')
df = df.explode('label_id')

# drop targets not in a pathway
df = df.loc[df.label_id != 'none']

df.shape

In [ ]:
df.ensg_id.unique().shape, df.cluster_id.unique().shape, df.label_id.unique().shape

In [ ]:
ena = go_utils.calc_enrichment_pvals(df)

In [ ]:
# merge pathway names, sort, rename cluster columns
pathways = ground_truth_labels.load_kegg_pathways()
ena_final = (
    ena
    .loc[ena.corrected_pval < 0.05]
    .merge(
        pathways.groupby(['pathway_id']).first().reset_index()[['pathway_id', 'pathway_name']],
        left_on='label_id',
        right_on='pathway_id',
    )
    .sort_values(by=['cluster_id', 'corrected_pval'])
    .rename(columns={'cluster_id': 'cluster_id_leiden_res6.31_seed44', 'label_id': 'pathway_id'})
)
ena_final

In [ ]:
ena_final.to_csv(output_dir / ('%s-kegg-pathway-enrichment.csv' % timestamp()), index=False)

In [ ]:
sc.pl.umap(cwv.adata, color='cluster_id_leiden_res0.63_seed41', palette='tab10', alpha=0.5)

### Aside: plots of F1 score vs Leiden resolution (using k-clique analysis)

This is not used. Note that the curves of F1 score and cumulative precision look the same for max_clique of 2 and 3.

In [ ]:
corum_kclique = cwv.calculate_kclique_metrics(
    drop_largest=False, max_clique=3, n_random_states=3
)

corum_kclique_wo_largest = cwv.calculate_kclique_metrics(
    drop_largest=True, max_clique=3, n_random_states=3
)

In [ ]:
plt.figure(figsize=(8, 6))
sns.lineplot(
    data=corum_kclique, x='resolution', y='grand_f1_score', label='Grand F1 score'
)
sns.lineplot(
    data=corum_kclique_wo_largest, x='resolution', y='grand_f1_score', label='Grand F1 score (w/o largest)'
)

plt.gca().set(xscale='log')

In [ ]:
len(set(cwv.adata.obs.corum_labels.str.split(';').explode().values))

### Sankey plot comparing OC annotations to low-resolution Leiden clusters

In [ ]:
cwv.run_leiden(resolution=0.63, random_state=44)

# hard-coded label order for OC grade_23_annotation_set
ground_truth_label_order = [
    'nucleolus_fc_dfc',
    'nucleolus_gc',
    'nuclear_punctae',
    'nuclear_punctae, nucleoplasm',
    'nucleoplasm',
    'chromatin',
    'nuclear_membrane',
    'cytoplasmic, nucleoplasm',
    'cytoplasmic',
    #'cytoplasmic, small_aggregates',
    #'cytoplasmic, membrane',
    'membrane',
    'centrosome',
    'vesicles',
    'golgi',
    'er',
]

# for res=0.63, random_state=44, using grade_23_annotation_set
predicted_label_order = [str(d) for d in [ 6, 10, 1, 9, 8, 5, 4, 2, 3, 0, 7]]

sankey_colormap = cwv.plot_sankey(
    ground_truth_label='grade_23_annotation_set',
    predicted_label='leiden',
    ground_truth_label_order=ground_truth_label_order[::-1],
    predicted_label_order=predicted_label_order[::-1]
)
plt.savefig(
    output_dir / ('%s-clustering-performance--oc-sankey-at-res0.63-seed44-flipped-w-sets.pdf' % (timestamp(),)),
    bbox_inches='tight'
)

In [ ]:
# plot the target UMAP colored by the low-res clusters 
# (must be done manually because sc.pl.umap generates a figure that is rasterized)
cwv.run_leiden(resolution=0.63, random_state=44)
sc.tl.umap(cwv.adata, init_pos='spectral', min_dist=0, random_state=51)

sankey_colormap['none'] = '#999999'
umap_coords = cwv.adata.obsm['X_umap']
umap_colors = [sankey_colormap[cluster_id] for cluster_id in cwv.adata.obs.leiden]

fig, axs = plt.subplots(1, 1, figsize=(5, 5))
for cluster_id in sorted(cwv.adata.obs.leiden.unique()):
    mask = cwv.adata.obs.leiden == cluster_id
    plt.scatter(
        umap_coords[mask, 0], 
        umap_coords[mask, 1], 
        color=sankey_colormap[cluster_id], 
        alpha=0.7, 
        label=cluster_id
    )
plt.legend()

fig.savefig(
    output_dir / ('%s-clustering-performance--umap-at-res0.63-seed44.pdf' % (timestamp(),)),
    bbox_inches='tight'
)

In [ ]:
# plot the UMAP colored with a given leiden clustering
fig, axs = plt.subplots(1, 1, figsize=(5, 5))
cwv.run_leiden(resolution=25, random_state=40)
cwv.plot_umap(
    color_label='leiden', min_dist=0.0, ax=axs, init_pos='spectral', random_state=51, palette='tab20'
)

In [ ]:
# the most common labels in a leiden cluster
d = cwv.adata.obs.copy()
n = d.loc[d.leiden.isin(['19',])].grade_3_annotation.value_counts()
n.iloc[:5]

### Aside: inspect the KEGG pathways

In [ ]:
kegg_pathways = ground_truth_labels.load_kegg_pathways(drop_largest=3)

In [ ]:
kegg_pathways.pathway_name.value_counts().head(15)

In [ ]:
# targets in any pathway
len(set(cwv.adata.obs.uniprot_id).intersection(kegg_pathways.uniprot_id))

In [ ]:
# pathways with at least one target (excluding the largest pathways)
pathways_in_oc = kegg_pathways[
    kegg_pathways.uniprot_id.isin(cwv.adata.obs.uniprot_id) 
]
pathways_in_oc.shape

In [ ]:
_ = plt.hist(pathways_in_oc.uniprot_id.value_counts(), bins=30)

In [ ]:
pathways_in_oc.uniprot_id.value_counts().value_counts()

In [ ]:
# targets in only one pathway
(pathways_in_oc.uniprot_id.value_counts() == 1).sum()

In [ ]:
# number of pathway-unique targets in each pathway
n = pathways_in_oc.uniprot_id.value_counts()
(
    pathways_in_oc.loc[pathways_in_oc.uniprot_id.isin(n.loc[n == 1].index.values)]
    .pathway_name
    .value_counts()
    .head(10)
)

### Aside: inspect GO-slim annotations

In [ ]:
ocgoslim = ground_truth_labels.load_go_slim()

In [ ]:
ocgoslim.go_label.value_counts().head(10)

In [ ]:
ocgoslim = ocgoslim.loc[
    ~ocgoslim.go_label.isin(['cytosol', 'nucleus', 'nucleoplasm', 'cytoplasm'])
]

In [ ]:
# number of targets with only one annotation from each type/aspect
# aside: dropping the four most common annotations increases the count of targets
# with only one CC label from 208 to 455
for go_type in ocgoslim.go_type.unique():
    n = ocgoslim.loc[ocgoslim.go_type == go_type].uniprot_id.value_counts()
    print('%s: %s' % (go_type, (n == 1).sum()))

In [ ]:
(cwv.adata.obs.go_cellular_component != 'none').sum()

### Aside: Measure the kNN accuracy of PCA vs UMAP coordinates

In [ ]:
cwv.calculate_neighbors(n_neighbors=10, n_pcs=200, metric='euclidean')
sc.tl.umap(cwv.adata, init_pos='spectral', min_dist=0, n_components=2, random_state=51)

In [ ]:
plt.figure(figsize=(6, 6))
sc.pl.umap(
    cwv.adata, color='grade_3_annotation', palette='tab10', alpha=0.5, legend_loc=None, ax=plt.gca()
)

In [ ]:
n_neighbors = [3, 10]

num_shared_neighbors = np.zeros((len(n_neighbors), len(n_neighbors))).tolist()
fig, axs = plt.subplots(len(n_neighbors), len(n_neighbors), figsize=(10, 7.5))

for ind0, k0 in enumerate(n_neighbors):
    for ind1, k1 in enumerate(n_neighbors):
        ax = axs[ind0][ind1]
        
        X = cwv.adata.obsm['X_pca'].copy()
        nns = sklearn.neighbors.NearestNeighbors(
            n_neighbors=(k0 + 1), algorithm='brute', metric='correlation'
        )
        distances, indices_pca = nns.fit(X).kneighbors(X)

        X = cwv.adata.obsm['X_umap'].copy()
        nns = sklearn.neighbors.NearestNeighbors(
            n_neighbors=(k1 + 1), algorithm='brute', metric='euclidean'
        )
        distances_umap, indices_umap = nns.fit(X).kneighbors(X)
        
        # the number of shared neighbors for each target (subtract one for each target itself)
        num_shared_neighbors[ind0][ind1] = [
            len(set(indices_pca[ind, :]).intersection(indices_umap[ind, :])) - 1
            for ind in range(indices_pca.shape[0])
        ]

        counts, edges = np.histogram(
            num_shared_neighbors[ind0][ind1], bins=np.arange(0, max(n_neighbors)), density=True
        )
        ax.bar(edges[:-1], counts)
        ax.set_ylim([0, 1])
        ax.set_xticks(range(max(n_neighbors)))
        ax.set_yticks([0, .25, 0.5, 0.75, 1])

In [ ]:
dists = sklearn.metrics.pairwise_distances(cwv.adata.obsm['X_pca'], metric='correlation')
_ = plt.hist(1 - dists.flatten(), bins=100)